## Gaussian Discriminant Analysis

with varying covariance $\Sigma_1 \neq \Sigma_2 \neq \Sigma_3 ... \neq \Sigma_3$

In [1]:
import pandas as pd
import numpy as np

# To reset the printoptions
# np.set_printoptions(edgeitems=3,infstr='inf',linewidth=75, nanstr='nan', precision=8,suppress=False, threshold=1000, formatter=None)

np.set_printoptions(precision=4, suppress=True)

#np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### For multi-class label y ={0, 1, 2, ...}

In [2]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target
#y=(y>0).astype(int) 

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

In [5]:
class GaussianDiscAnalysis: 
    """
    Fits a Quadratic Discriminant Analysis Model for multi-class data where covariance for each class are unique.
    
    lamd : Preconditioner; for matrix inversion; default:1e-10
    """
    def __init__(self, lambd=1e-10):
        self.lambd = lambd
        
    def compute_phi(self, y):
        n = len(y)
        phi = dict()
        for idx in range(len(self.classes)):
            phi[idx] = (1/n) * np.sum(y==idx)
        return phi
    
    def compute_mu(self, X, y):
        mu_dict = dict()
        for idx in range(len(self.classes)):
            # Add mu for each class
            mu_dict[idx] = np.sum(X[y==idx], axis=0)/ np.sum(y==idx)
        return mu_dict

    def compute_sigma(self, X, y):
        d = X.shape[1]
        sigma_dict = dict()
        Xmu = X.copy()
        for idx in range(len(self.classes)):
            n = np.sum(y==idx)
            Xmu_i = Xmu[y==idx] - self.mu[idx]
            sigma = (1/n) * Xmu_i.T@Xmu_i
            sigma_dict[idx] = sigma + self.lambd*np.eye(d,d) # Precondition sigma
        return sigma_dict
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        m = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma[idx])
        det_sigma = np.linalg.det(self.sigma[idx])
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(m/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        d = X.shape[1]
        self.classes = np.unique(y)
        self.mu = self.compute_mu(X, y)
        self.sigma = self.compute_sigma(X, y)
        self.phi = self.compute_phi(y)
        
    def predict_proba(self, X):
        """Computes the probability of example belonging to that class"""
        n = len(X)
        Pyi = np.zeros((n, len(self.classes)))
        
        for idx in range(len(self.classes)):
            #print(self.compute_Pxyi(X, idx))
            py_i = self.compute_Pxyi(X, idx) * self.phi[idx]
            Pyi[:, idx] = py_i
        return Pyi
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def generate_data(self, class_id, num_samples=1):
        """Generates new unseen dataset from a normal distribution
            given the mean of class and covariance
        """
        mean = self.mu[class_id]
        cov = self.sigma[class_id]
        return np.random.multivariate_normal(mean, cov, num_samples)

In [6]:
GDA = GaussianDiscAnalysis()

In [7]:
GDA.fit(X_train, y_train)

In [8]:
predictions = GDA.predict(X_test)

In [9]:
predictions

array([1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 0, 1, 0, 0, 1, 2, 1, 0, 1, 0, 1, 2,
       1, 0, 1, 2, 0, 0, 2, 1, 1, 2, 2, 2, 0, 1, 2, 0])

In [10]:
GDA.predict_proba(X_test)[0:5]

array([[0.    , 0.0939, 0.0282],
       [0.    , 0.4907, 0.0013],
       [0.    , 0.2521, 0.    ],
       [0.    , 0.008 , 0.1905],
       [0.    , 0.0004, 0.1725]])

In [11]:
np.sum(predictions == y_test) / len(y_test)

0.9210526315789473

### Generate new data

In [12]:
new_data = GDA.generate_data(class_id=1, num_samples=20)

In [13]:
new_data

array([[5.6335, 2.2214, 3.9456, 0.9979],
       [6.523 , 3.4725, 4.6979, 1.5124],
       [6.3917, 3.2641, 4.8062, 1.5084],
       [4.702 , 2.4147, 4.1388, 1.3575],
       [6.2298, 2.8071, 4.5052, 1.4973],
       [5.8649, 3.0309, 4.4115, 1.3666],
       [6.0956, 3.0507, 4.7172, 1.5248],
       [5.9547, 2.9497, 4.6106, 1.6344],
       [5.749 , 2.5659, 4.0129, 1.117 ],
       [4.8475, 2.8935, 3.1269, 1.1904],
       [6.1344, 2.6237, 4.2573, 1.2619],
       [5.8221, 2.9997, 4.6691, 1.6195],
       [6.3808, 2.6025, 4.3131, 1.3017],
       [5.6269, 2.8229, 4.3459, 1.4423],
       [5.5626, 2.6335, 4.6698, 1.506 ],
       [5.6422, 2.7802, 3.8655, 1.2853],
       [6.3596, 3.074 , 4.5231, 1.3924],
       [5.849 , 2.6853, 3.7337, 1.2429],
       [4.9856, 2.5777, 4.1021, 1.3484],
       [6.5341, 3.5588, 5.4665, 1.8618]])

In [14]:
GDA.predict(new_data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2])